[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aprendizaje-automatico-dc-uba-ar/material/blob/main/notebooks/notebook_05_seleccion_modelos-published.ipynb)

# Selección de modelos

## Cross validation 

Hasta ahora sólo habíamos visto (ver en el [notebook 03](https://github.com/aprendizaje-automatico-dc-uba-ar/material/blob/main/notebooks/notebook_03_arboles_de_decision_sklearn-published.ipynb)) que ibamos a dividir los datos en train y test.


En esta semana vimos la opción de hacer validación cruzada. En esta oportunidad lo que haremos sera realizar una exploración de hiperparámetros para árboles incorporando conceptos de la clase de esta semana.
Vamos a experimentar usando k-fold (con k=10) para explorar distintos valores de configuración de `DecisionTreeClassifier` para seleccionar el hiperparámetro que nos parezca el mejor. 
Ensayaremos áltura máxima con valores `[None, 1, 2, 3, 5, 8, 13, 21]`. 

Nos interesará:
- controlar el tiempo de entrenamiento
- generar alguna métrica que elijamos para seleccionar la áltura máxima

Con la mejor configuración obtenida entrenar un clasificador con todos los datos de desarrollo.
    
Evaluar el comportamiento con el set de evaluación
    



Primero separaremos nuestro data set entre **desarrollo** y **evaluación** en un 10%. Para esto podemos usar `train_test_split`

In [ ]:
print("Longitud original:", len(y))
print("Longitud del set de desarrollo:", len(y_dev))
print("Longitud del set de evaluación:", len(y_eval))

In [ ]:
def train_tree(X_tr: np.ndarray, y_tr: np.ndarray, tree_params={}) -> DecisionTreeClassifier:
    arbol = DecisionTreeClassifier(**tree_params) #crea el arbol con ciertos hiperparametros que le pasas: la altura maxima 
    arbol.fit(X_tr, y_tr)

    return arbol

def tree_predict(ab: DecisionTreeClassifier, X_test: np.ndarray) -> np.ndarray:
    predictions = ab.predict(X_test) #le pasas el arbol ya entrenado y te devuelve las predicciones del test
    return predictions

In [ ]:
def accuracy(y_predicted: np.ndarray, y_real: np.ndarray) -> float:
    TP_TN = sum([y_i == y_j for (y_i, y_j) in zip(y_predicted, y_real)]) 
    P_N = len(y_real)
    return TP_TN /P_N

def precision_recall(y_predicted: np.ndarray, y_real: np.ndarray) -> np.ndarray:
    TP = sum([y_i == 1 and y_j == 1 for (y_i, y_j) in zip(y_predicted, y_real)])
    FP = sum([y_i == 1 and y_j == 0 for (y_i, y_j) in zip(y_predicted, y_real)])
    FN = sum([y_i == 0 and y_j == 1 for (y_i, y_j) in zip(y_predicted, y_real)])
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    return precision, recall

def f_score(y_predicted: np.ndarray, y_real: np.ndarray, beta = 0.5) -> float:
    presicion, recall = precision_recall(y_predicted, y_real)
    f_score = (1 + beta**2) * (presicion * recall) / ((beta**2) * presicion + recall)
    return f_score

def metrica_seleccionada(y_predicted: np.ndarray, y_real: np.ndarray) -> float:
    return accuracy(y_predicted, y_real)

Realización del experimento.

Nota: se inicializa con una semilla para poder reproducir el resultado.

In [ ]:
results = []

np.random.seed(44)
for h_max in [None, 1, 2, 3, 5, 8, 13, 21]:
    kf = KFold(n_splits=10) #voy a usar 10 folds
    y_pred = np.empty(y_dev.shape)
    y_pred.fill(np.nan)
    
    # generamos para cada fold una predicción
    for train_index, test_index in kf.split(X_dev): 
        #en train_index estan los indices de los datos que estan en los 9 folds que pertenecen a training
        #en test_index estan los indices de los datos que estan en el unico fold que es el test
        
        #saco el fold que no uso para entrenar
        kf_X_train, kf_X_test = X_dev[train_index], X_dev[test_index]
        kf_y_train, kf_y_test = y_dev[train_index], y_dev[test_index]

        current_tree = train_tree(kf_X_train, kf_y_train,
                                    tree_params={"max_depth":h_max})
        predictions = tree_predict(current_tree, kf_X_test)
        y_pred[test_index] = predictions #quedan algunos vacios (con NAs) en cada iteracion pero finalmente se llena todo
        
    current_score = metrica_seleccionada(y_pred, y_dev) #mido que tan bien me fue con las predicciones
    
    results.append((h_max,current_score)) #para cada altura se guarda la performance
    

# Ordenamos los resultados (puede ser que convenga del derecho o del reves) por score de mayor a menor
r = sorted(results, key=lambda x: x[1], reverse=True)

print("Órden obtenido según la métrica elegida")
for idx, (h, sc) in enumerate(r):
    print(f"\t{idx+1}- h_max={h} con {sc:.3f}")


In [ ]:
# elegimos la altura 1 porque es el que más F-score tuvo
h_max = r[0][0] 
selection_score = r[0][1] 
print(h_max)
print(selection_score)

In [65]:
print(f"Construimos nuestro clasificador con parámetro 'max_depth'={h_max}."
     + f"\nPara seleccionarlo el score que habíamos obtenido era {selection_score:.3f}")

best_tree = train_tree(X_dev, y_dev,
                            tree_params={"max_depth":h_max})


Construimos nuestro clasificador con parámetro 'max_depth'=1.
Para seleccionarlo el score que habíamos obtenido era 0.881


In [66]:
y_pred = tree_predict(best_tree, X_dev)
best_tree_score = metrica_seleccionada(y_pred, y_dev)
print(f"El F-score del árbol seleccionado es {best_tree_score:.3f}")

El F-score del árbol seleccionado es 0.904


¿Qué nos están diciendo estos dos scores?¿Para qué nos sirven?

In [70]:
y_pred_eval = tree_predict(best_tree, X_eval)       
best_tree_score_eval = metrica_seleccionada(y_pred_eval, y_eval)

print(f"Con el árbol entrenado con el parámetro seleccionado tenemos en eval un score de {best_tree_score_eval:.3f}")

Con el árbol entrenado con el parámetro seleccionado tenemos en eval un score de 0.733


## Opcionales

1. Simular qué hubiese ocurrido si hubieramos elegido un K distinto. ¿La diferencia entre el score en *dev* y el score en *eval* cambia significativamente?
2. Repetir el mismo ejercicio de elegir la mejor combinación de parametros pero esta vez establecer una grilla donde se exploren al menos dos hiperparámetros que no sean la altura máxima. Revisar la documentación de `DecisionTreeClassifier`, por ejemplo pueden elegir la **medida de impureza** y el **mínimo de muestas necesario para realizar un split**. Definir los rangos necesarios para explorar más de un valor de cada hiperparámetro considerado. ¿Este modelo fue mejor que el obtenido en el punto anterior?

**Importante**: en este punto nos tomamos la licencia de usar nuevamente el conjunto de evaluación. El re-uso de el conjunto de evaluación sólo lo permitimos en este caso por motivos pedagócios. Pero **NO DEBE** suceder en la práctica.

